### Buidling with langchain

**Modules:** LangChain offers diverse modules for building language model applications. These modules work both as standalone entities and in combination for complex tasks.

**LCEL:** Composition is enabled by the LangChain Expression Language, which allows for the integration of various modules through a unified Runnable interface.

Core Components:

- **LLM/Chat Model:** Fundamental reasoning engine. Understanding different language models is crucial.
- **Prompt Template:** Key for directing the language model's output. Skills in prompt construction and strategies are essential.
- **Output Parser:** Transforms raw language model responses into usable formats.
- **Customization:** Most LangChain applications allow configuration of the model and prompts, important for effective use.

### LLM vs. Chat Model

1. **Types of Language Models**:
   - **LLM (Language Learning Model)**: Takes a string as input and returns a string.
   - **ChatModel**: Accepts a list of messages as input and returns a message.

2. **Message Types**:
   - **BaseMessage Interface**: Has two required attributes:
     - `content`: The message content, usually a string.
     - `role`: The source of the `BaseMessage`.
   - **Specialized Message Types**:
     - **`HumanMessage`**: Originates from a human/user.
     - **`AIMessage`**: Comes from an AI/assistant.
     - **`SystemMessage`**: Generated by the system.
     - **`FunctionMessage` / `ToolMessage`**: Contains the output from a function or tool.
     - **`ChatMessage`**: Allows manual specification of the role.

3. **Common Interface**:
   - Both LLMs and ChatModels share a common interface, crucial for constructing appropriate prompts.

4. **Method Invocation**:
   - **`LLM.invoke()`**: Input a string, returns a string.
   - **`ChatModel.invoke()`**: Input a list of `BaseMessage`, returns a `BaseMessage`.

5. **Usage Example**:
   - Import and create LLM and ChatModel objects.
   - Initialize with parameters like temperature for custom configurations.
   - Sample Invocation:
     - LLM example: Input a text string, returns a string (e.g., a company name suggestion).
     - ChatModel example: Input a list of messages (e.g., `HumanMessage`), returns an `AIMessage` with a response.


In [4]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

In [5]:
llm = OpenAI()
chat_model = ChatOpenAI()

In [7]:
text = """
What would be a good company name for a company making colorful socks?
"""

messages = [HumanMessage(content=text)]

#### LLM example

In [8]:
response = llm.invoke(text)
print(response)


Bright Toes Socks


#### Chat Model Example

In [9]:
chat_model.invoke(messages)

AIMessage(content='Rainbow Feet')

### Prompt templates

1. **Role of Prompt Templates**:
   - LLM applications typically use prompt templates instead of directly passing user input to the LLM. These templates combine user input with additional context-relevant text for specific tasks.

2. **Basic Functionality**:
   - Prompt templates simplify user interactions by handling the generation of detailed instructions for the model. An example is creating a prompt for naming a company based on a product description.

3. **Advantages**:
   - Allows partial formatting, where only some variables are formatted at a time.
   - Supports composition, enabling the combination of multiple templates into a single prompt.
   - Offers flexibility and advanced formatting options, detailed in the section on prompts.

4. **ChatPromptTemplate**:
   - Generates a list of messages, each with specific roles and content, suitable for chat interactions.
   - An example usage involves creating a sequence of messages like `SystemMessage` and `HumanMessage`, each with formatted content based on the template.

5. **ChatPromptTemplate Customization**:
   - Can be constructed in various ways to suit different use cases, with more detailed explanations available in the prompts section.


In [18]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

In [15]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks'

In [28]:
system_template = "You are a helpful assistant that translates {input_language} to {output_language}"
human_template = "{text}"

chat_prompt_v1 = ChatPromptTemplate.from_messages([
  ("system", system_template),
  ("human", human_template)
])


# Same thing as above - Just for illustration
chat_prompt_v2 = ChatPromptTemplate.from_messages(
  [
    SystemMessage(content=system_template),
    HumanMessage(content=human_template)
  ]
)

In [29]:
chat_prompt_v1.format_messages(
  input_language="English",
  output_language="French",
  text="I Love Programming"
  
  )

[SystemMessage(content='You are a helpful assistant that translates English to French'),
 HumanMessage(content='I Love Programming')]

In [30]:
chat_prompt_v2.format_messages(
  input_language="English",
  output_language="French",
  text="I Love Programming"
  
  )

[SystemMessage(content='You are a helpful assistant that translates {input_language} to {output_language}'),
 HumanMessage(content='{text}')]

### Output parsers

- **Purpose of OutputParsers**:
  OutputParsers convert the raw output from a language model into a usable format for downstream applications.

- **Main Types**:
  - Convert text from LLM into structured formats like JSON.
  - Change a `ChatMessage` into a plain string.
  - Transform additional data from model calls (e.g., OpenAI function invocation) into a string.

- **Custom OutputParser Example**:
  An example in the guide shows creating a custom OutputParser, `CommaSeparatedListOutputParser`, which turns a comma-separated string into a Python list. It's derived from `BaseOutputParser`, with a `parse` method that splits a string input into a list.

- **Practical Use**:
  This parser is useful for converting outputs like "hi, bye" into a list format `['hi', 'bye']`.


In [32]:
from langchain.schema import BaseOutputParser

In [33]:
class CommaSeparatedListOutputParser(BaseOutputParser):
  """
  Parse the putput of an LLM call to a comma-separated list
  """

  def parse(self, text: str):
    return text.replace(" ", "").split(",")

In [36]:
CommaSeparatedListOutputParser().parse("hi, bye, bye, bye")

['hi', 'bye', 'bye', 'bye']

### Composing with LCEL


- **Combining Components into a Chain**:
  LangChain allows for combining various components into a single chain. This chain process involves taking input variables, creating a prompt via a prompt template, passing the prompt to a language model, and optionally processing the output with an output parser.

- **Example Implementation**:
  - A custom `CommaSeparatedListOutputParser` is defined to parse the language model's output into a comma-separated list.
  - A `ChatPromptTemplate` is created with a system message template for generating lists and a human message template for user input.
  - These components are combined into a chain: `chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()`.
  - Invoking the chain with an input like `{"text": "colors"}` results in an output such as `['red', 'blue', 'green', 'yellow', 'orange']`.

- **Using the Pipe (|) Syntax**:
  - The pipe (`|`) syntax, part of the LangChain Expression Language (LCEL), is used for joining these components. It leverages the universal Runnable interface that all these objects implement.


In [37]:
system_template = """
You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more.
"""

human_template = "{text}"

chat_prompt_v1 = ChatPromptTemplate.from_messages(
  [
    ("system", system_template),
    ("human", human_template)
  ]
)

chat_prompt_v2 = ChatPromptTemplate.from_messages(
  [
    SystemMessage(content=system_template),
    HumanMessage(content=human_template)
  ]
)

In [38]:

chain_v1 = chat_prompt_v1 | ChatOpenAI() | CommaSeparatedListOutputParser()
chain_v2 = chat_prompt_v2 | ChatOpenAI() | CommaSeparatedListOutputParser() 

In [43]:
chain_v1.invoke({"text": "vehicles"})

['car', 'truck', 'motorcycle', 'bicycle', 'bus']